An Example Notebook for doc_search
==============

This notebook provides an example of using the doc_search widget and download to identify documents associated with universities. If you are experienced with using Jupyter Notebooks it should be straighforward to work through this and run it. Dependencies are noted in the package README.md and requirements.txt

If you are less familiar with Jupyter notebooks but have successfully got to this point where you are reading this in a web browser then you are on the right track. Work your way through the notebook changing anything you want to modify. This will mostly be the paths to files, both input and output, and your search term. Instructions are provided as you go. Each time you want to run a cell you need to select it and type shift-return.

When you use this for your own work you will probably want to make a copy of this notebook and take your own notes on what you are doing. There are lots of good resources on using Jupyter Notebooks which can be found online.

First steps
-----------

The very first step you will need is to obtain a Bing API key for running the searches. For small scale work and testing you should be able to get a new account with a reasonable number of free calls. If you scale up you may find you need to pay a small amount but most of our work hasn't hit that threshold. You can get an API key at: https://azure.microsoft.com/en-us/services/cognitive-services/bing-web-search-api/ and select the 'Try Bing Web' option and then 'Free Azure Account'. You should ultimately get a key, which needs to be added at the top of the file doc_search/doc_search/search.py on the line that says:

`subscriptionKey = "subscriptionKey" <---change the second bit with your key that looks like 87dg7654has237`

In this notebook we first need to set up some variables to tell the tool where to find important files. You will need to modify the unilistfile to include the GRID IDs for your target universities. The output directory will hold both downloaded files and a catalog file that records your work.  

In [ ]:
unilistfile = 'example_uni_list.csv' # You will need to modify this file for your target universities!!
outputdir = 'data/output/' # This default creates a directory for your outputs
outputfile = 'NAME_OF_OUTPUT_FILE.csv' # Change this !!
griddir = 'doc_search/grid/' # This shouldn't need to change unless you moved that directory

Setting up the search term
-------------------------------

Then we set up the search term. There are a range of options here and the full set of Bing advanced search operators should be available. A particularly useful option is filetype:pdf for pdf documents. Any text that needs to be searched for exactly should be enclosed in double quotes as for "library access policy" below. 

Note that by default the search will be limited to the URLs that GRID is aware of for the university. If your target documents are not on the university domain they may not be found. if you want to turn this off then change the second line in the cell below to read as follows:

`includeuniurl = False`

In addition to the Bing options it is also possible to include elements from the GRID data for the university. These need to be included inside a pair of curly braces as shown for {name} below which will insert the university's name from the GRID data. Only those elements of the GRID JSON with a top level text entry will provide sensible results. These options include:
* {name} - the university's full name
* {id} - university's GRID ID
* {wikipedia_url} - university's wikipedia URL
* {established} - the date of the university's founding

In [ ]:
searchterm = '{name} "library access policy"'
includeuniurl = True

Don't worry about this next cell unless you understand this bit
--------------------------------------------------

Then we have a few magical incantations to load helper functions and make sure everything is working. 

In [ ]:
from doc_search import doc_search, widgets
import requests, csv, os, os.path, datetime, urllib.error

The search widget
-----------------

This is the main part of the notebook. When you execute this cell a window should appear with search results for your first university. There may be up to 5 results depending on what Bing finds. You can select as many of the results as you wish using the buttons below the window. Clicking again will clear a button and hitting 'None' will clear all the buttons. 

'Next' and 'Previous' move to the next and previous university respectively. Behaviour with the 'Previous' button is not always exactly as expected so avoid unless necessary. It should however be possible to just keep adding to the output file which should not overwrite previous results. The progress bar will tell you how far through a set of results you are. 

In [ ]:
outfilepath = os.path.join(outputdir, outputfile)
m = doc_search.Model(unilistfile = unilistfile,
                     searchterm = searchterm,
                     outfilepath = outfilepath,
                     includeuniurl = includeuniurl,
                     grid_dir = griddir
                    )
w = widgets.SelectionWidget(model=m)

Downloading
-----------

Once you have identified all the files or websites you feel are correct you can then proceed to download them. This is kept separate from the search process to reduce any problems that might arise. You can run the download repeatedly as it will only download the files it doesn't yet have. Files will be named with university name, the search term and the search rank. You can change the search term and run your search again if desired.

In [ ]:
if not os.path.isdir(outputdir):
    os.makedirs(outputdir)

cat = doc_search.Model(outfilepath = outfilepath)
cat.load_catalog()

for uni in cat.catalog:
    if uni['searchrank'] != '':
        filename = '{}_{}_{}'.format(uni['name'].replace(' ', '_'),
                                     uni['searchterm'].replace(' ', '_').replace(':', '_'),
                                     uni['searchrank'])
        filepath = os.path.join(outputdir,filename)

        if os.path.isfile(filepath) or os.path.isfile(filepath + '.html') or os.path.isfile(filepath + '.pdf'):
            continue
        try:
            print(uni['name'], uni['url'])
            resp = requests.get(uni['url'], headers = {'User-Agent': 'Mozilla/5.0'})
            with open(filepath, 'wb') as f:
                f.write(resp.content)
            #localfilename, headers = urllib.request.urlretrieve(url, filepath)
            uni['download_file'] = filename
            uni['download_datetime'] = datetime.datetime.utcnow().isoformat()

        except (urllib.error.HTTPError, urllib.error.URLError, requests.exceptions.SSLError):
            print('\nFailed to download from:', url, '\n')
            
cat.save_full()